In [1]:
# Start writing code here...
import numpy as np 
import pandas as pd
  
from IPython.display import display, clear_output
    
from tqdm import tqdm

from ast import literal_eval

from transformers import AutoTokenizer, AutoModel

In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1407058959519654060,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15210480960
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15725232135477842131
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
!pip install --no-index --find-links "../input/spacy3" spacy[cuda110]
!pip install --no-index --find-links "../input/spacy3" en_core_web_trf
!pip install --no-index --find-links "../input/spacy3" spacy_transformers

Looking in links: ../input/spacy3
Looking in links: ../input/spacy3
Processing /kaggle/input/spacy3/en_core_web_trf-3.0.0-py3-none-any.whl
Processing /kaggle/input/spacy3/spacy_transformers-1.0.2-py2.py3-none-any.whl
Processing /kaggle/input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/typer-0.3.2-py3-none-any.whl
Processing /kaggle/input/spacy3/pathy-0.5.2-py3-none-any.whl
Processing /kaggle/input/spacy3/catalogue-2.0.3-py3-none-any.whl
Processing /kaggle/input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl
Processing /kaggle/input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/smart_open-3.0.0-py3-none-any.whl
Processing /kaggle/input/spacy3/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl
  Attempting uninstall: catalogue
    Found 

In [4]:
import spacy
from spacy.tokens import DocBin

In [5]:
print(spacy.__version__)

3.0.6


In [6]:
%env TRANSFORMERS_OFFLINE=1
%env HF_DATASETS_OFFLINE=1

env: TRANSFORMERS_OFFLINE=1
env: HF_DATASETS_OFFLINE=1


In [7]:
# Load pre-existing spacy model
try:
    spacy.require_gpu()
except:
    print("GPU not found")

nlp = spacy.load('en_core_web_trf', disable=["tagger","parser","attribute_ruler","lemmatizer"])

In [8]:
name = "../input/scibert-scivocab-uncased"
local_path = "./scibert-scivocab-uncased"
model = AutoModel.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)

# add the setting (note that you can modify tokenizer.model_max_length on the fly, 
# but frustratingly this change isn't saved as part of the saved config)
tokenizer.init_kwargs["model_max_length"] = 512

# save
tokenizer.save_pretrained(local_path)
model.save_pretrained(local_path)

In [9]:
train = pd.read_csv("../input/colerigde-processed-text/train.csv",index_col=0)
validation = pd.read_csv("../input/colerigde-processed-text/validation.csv",index_col=0)

train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

In [10]:
train['dataset_index'] = train['dataset_index'].apply(lambda x: literal_eval(x))
validation['dataset_index'] = validation['dataset_index'].apply(lambda x: literal_eval(x))

In [11]:
print(train.shape)
train.head()

(25686, 5)


,Section,Text,Label,Id,dataset_index
0,Results of the Cochrane review,The highly heterogeneous [\n18 F]FDG PET sensi...,ADNI,75c5eff6-27c1-4fd3-9d35-68e48799e6cd,"[(1986, 1990, DATASET), (2267, 2271, DATASET)]"
1,Cognitive abilities,Prior content knowledge Since students possess...,Trends in International Mathematics and Scienc...,d24cb86b-bfe6-4f58-8e5d-0187f8e5681d,"[(1270, 1323, DATASET)]"
2,Overview,The Baccalaureate and Beyond Longitudinal Stud...,Baccalaureate and Beyond Longitudinal Study,c7f1f747-bc8e-4de6-a993-9ef9f54bdbf7,"[(4, 47, DATASET), (1013, 1056, DATASET), (116..."
3,Fourth-Grade Stratification,The following tables show the total number and...,Trends in International Mathematics and Scienc...,85de886e-eb37-4402-b219-386686cfc493,"[(759, 812, DATASET)]"
4,Introduction,"ey have used the OASIS dataset for experiment,...",ADNI,878d4d2e-020a-48f8-854f-81894819c770,"[(1468, 1472, DATASET), (1775, 1779, DATASET)]"


In [12]:
print(validation.shape)
validation.head()

(2854, 5)


,Section,Text,Label,Id,dataset_index
0,Abstract,A rterial stiffness is an age-related trait th...,Baltimore Longitudinal Study of Aging,58147ea8-c5c7-4283-bb27-c01830c3af13,"[(1309, 1346, DATASET), (2525, 2563, DATASET)]"
1,Introduction,"(2012 Jedynak et al. ( , 2015 proposed a param...",Alzheimer's Disease Neuroimaging Initiative (A...,1c35d627-8b39-41d2-b3e0-77c92483d8a8,"[(1925, 1975, DATASET)]"
2,Impact of Intensification Rate on TC Intensity...,[11] How do the changes in intensification rat...,IBTrACS,5716fd22-6c6c-4587-a673-281a33456749,"[(334, 341, DATASET)]"
3,Introduction,Sponsored by the National Center for Education...,Early Childhood Longitudinal Study,3be437b7-2576-4b2f-ae22-d158cf97edfe,"[(59, 93, DATASET)]"
4,Introduction,"(2010) , we developed a novel surface mapping ...",Alzheimer's Disease Neuroimaging Initiative (A...,6b460753-749f-45bc-9996-dd8a950107ce,"[(1787, 1837, DATASET)]"


In [13]:
train["dataset_index"][0]

[(1986, 1990, 'DATASET'), (2267, 2271, 'DATASET')]

In [14]:
TRAIN_DATA = []
VALIDATION_DATA = []

for index, row in train.iterrows():
    TRAIN_DATA.append((row["Text"],{"entities":row["dataset_index"]}))
    
for index, row in validation.iterrows():
    VALIDATION_DATA.append((row["Text"],{"entities":row["dataset_index"]}))
    
print(TRAIN_DATA[10])
print(VALIDATION_DATA[10])

("Thickness and vertical height of oyster aggregations are subdivided, and oysters are classified as solid or scattered along the substrate: = Light: 1 or 2 layers of oysters in a band less than 15 cm (6 in) wide = Medium: more than 1 layer of oysters in a band 15-30 cm (6-12 in) wide = Heavy: more than 1 layer of oysters in a band 30-46 cm (12-18 in) wide = Very heavy: more than 1 layer of oysters in a band >46 cm (18 in) wide < Solid: solid stretch of oysters along the seawall or riprap < Scattered: sporadic stretch of oysters along the seawall or riprap The South Carolina Department of Natural Resources (SCDNR) and the NOAA Coastal Services Center mapped the coastline of South Carolina using multispectral imagery (SCDNR 2008). The effort included the development of classification system for oyster reefs, largely using different spectral signatures as a result of vertical or horizontal orientation of the oysters. The project report (SCDNR 2008) describes each category with accompanyi

In [15]:
# Convert SpaCy v2 to SpaCy v3 object

db = DocBin() # create a DocBin object
i = 1

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            i += 1
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

print("Skipped "+ str(i) +" entities")
    
db.to_disk("train.spacy") # save the docbin object

100%|██████████| 25686/25686 [01:20<00:00, 320.01it/s]


Skipped 14 entities


In [16]:
db = DocBin() # create a DocBin object
i = 1

for text, annot in tqdm(VALIDATION_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            i += 1
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

print("Skipped "+ str(i) +" entities")
    
db.to_disk("validation.spacy") # save the docbin object

100%|██████████| 2854/2854 [00:08<00:00, 340.43it/s]


Skipped 2 entities


In [17]:
!python -m spacy init config "base_config.cfg" --lang="en" --pipeline=["transformer","ner"] --optimize="accuracy" --gpu --force

2021-06-13 16:06:48.216758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: GPU
- Transformer: roberta-base
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [18]:
import configparser

# Read config file
edit = configparser.ConfigParser()
edit.read("./base_config.cfg")

arch = edit['components.transformer.model']
arch["name"] = '"'+local_path+'"'

edit.remove_option('components.ner.model', 'no')

edit.remove_option('training.optimizer', 'l2')
edit.remove_option('training.optimizer', 'l2_is_weight_decay')

# Write changes back to file
with open('./base_config.cfg', 'w') as configfile:
    edit.write(configfile)

In [19]:
!python -m spacy init fill-config "base_config.cfg" "config.cfg"

2021-06-13 16:07:02.611033: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [20]:
!python -m spacy train "config.cfg" \
    --output "./output" \
    --paths.train "train.spacy" \
    --paths.dev "validation.spacy" \
    --gpu-id 0 \
    --nlp.batch_size 64 \
    --training.dropout 0.2 \
    --training.patience 0 \
    --training.max_steps 5000 \
    --training.logger.progress_bar True

2021-06-13 16:07:12.770703: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Created output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-06-13 16:07:17,477] [INFO] Set up nlp object from config
[2021-06-13 16:07:17,492] [INFO] Pipeline: ['transformer', 'ner']
[2021-06-13 16:07:17,498] [INFO] Created vocabulary
[2021-06-13 16:07:17,498] [INFO] Finished initializing nlp object
[2021-06-13 16:09:59,533] [INFO] Initialized pipeline components: ['transformer', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        4897.79   1169.12    0.05    0.03    1.75    0.00
  0     200       9

In [21]:
# Load the best model
spacy.require_gpu()
nlp_best = spacy.load(R"./output/model-best")

In [22]:
# Testing the model
doc = nlp_best('The supply of PCR reagents, trained lab personnel and the availability of laboratories with sufficient biocontainment levels are major challenges of SARS-CoV-2 detection in developing countries, such as Indonesia (Younes et al., 2020) . Therefore, it is not surprising that the tested people per week is still lower than the World Health Organization (WHO) standard (World Health Organization, 2020b) . Recently, SARS-CoV-2 with the D614G mutation became the most frequently detected globally, including South East Asia region (Korber et al., 2020; Nguyen et al., 2020) . Interestingly, SARS-CoV-2 with the G614 variant had significantly higher infectious titers than the original D614 virus, and COVID-19 patients with the G614 variant had a higher viral load than patients without the mutation (Korber et al., 2020) . A recent study showed that the SARS-CoV-2 with the G614 variant revealed increased infectivity, competitive fitness, and transmission than the wild-type D614 virus in human airway epithelial cells and hamster (Hou et al., 2020) . However, this mutation was not associated with the severity of COVID-19 (Korber et al., 2020; Nguyen et al., 2020) . Here, we aimed:\n(1) to report full-length genome sequences of SARS-CoV-2 collected from four COVID-19 patients in the Special Region of Yogyakarta and Central Java provinces, Indonesia;\n(2) to compare the clade distribution of full-length genome sequences from Indonesia (n = 60) from March to September 2020; and (3) to perform phylogenetic analysis of SARS-CoV-2 complete genomes from different countries, including Indonesia.') # input sample text
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('genome sequences of SARS-CoV-2', 'DATASET')]


In [23]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter